<a href="https://colab.research.google.com/github/reevafaisal/bluesky/blob/main/BlueSky.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**UPDATES:**
- Drive setup code is saved in GitHub repo as *`drive_setup.py`*

**NOTE:**
- After every work session please push changes to the github repo to save work history: https://github.com/reevafaisal/bluesky

Run the cell below for all the required packages and installations.

In [ ]:
!pip install tqdm
!pip install atproto
!pip install atproto pandas requests tqdm
from google.colab import drive
from google.colab import drive
import os
import sys
import tarfile
from tqdm import tqdm
import os
import tarfile
from atproto import Client
import unicodedata
import pandas as pd
from requests.exceptions import RequestException
import datetime
from requests.exceptions import HTTPError

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 351.1/351.1 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 671.7/671.7 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 8.3 MB/s eta 0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 14.2
    Uninstalling websockets-14.2:
      Successfully uninstalled websockets-14.2
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1


- **NOTE: do not forcibly remount, only uncomment and run if drive gets disconnected**

In [ ]:
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
drive_path = "/content/drive/My Drive/ZenodoFiles/scripts.tar.gz"
extract_path = "/scripts"

if not os.path.exists(extract_path):
    os.makedirs(extract_path)

file_name = os.path.join(extract_path, "scripts", "data_collection", "crawl_feed_post_likes.py")

# The code is just for displaying file content in order to refer to the original if needed

# if os.path.exists(file_name):
#     with open(file_name, "r") as f:
#         content = f.read()
#     print("\nFile Content:\n", content)
# else:
#     print(f"\nError: '{file_name}' not found! Check the extracted files above and adjust the path.")

Original `crawl_feed_bookmarks.py` script uses `otherfile.py` containing client information and the myfeeduris dictionary. Since it is not available in the publicly available file set, here are the methods used:

*   Account information:
  *   username: blueskyre.bsky.social
  *   password: blueskyresearch

*  `myfeeduris` (`dict`):
  * first extracted all unique uris from entire feed.
  * loaded them into a dict called `myfeeduris`



In [6]:
# Function to initialize the Bluesky API client
def init_client():
    client = Client()
    try:
        client.login("blueskyre.bsky.social", "blueskyresearch")
        print("✅ Successfully authenticated to Bluesky API")
    except Exception as e:
        print(f"❌ Failed to authenticate: {e}")
        return None
    return client

import unicodedata

def normalize_uri(uri):
    """Normalize URIs: remove spaces, convert to lowercase, and normalize Unicode."""
    return unicodedata.normalize("NFKC", uri.strip().lower())

client = init_client()
# print(client)

feeds = client.app.bsky.unspecced.get_popular_feed_generators()

unique_uris = set()
for f in feeds.feeds:
    unique_uris.add(normalize_uri(f.uri))

myfeeduris = {f"Feed_{i+1}": uri for i, uri in enumerate(unique_uris)}
# print(myfeeduris)

✅ Successfully authenticated to Bluesky API


In [ ]:
def collect_likes(client, uri, cursor=None, likes=None):
    cursor = None
    old_cursor = None

    if likes is None:
        likes = []


    while True:

        try:
            fetched = client.get_likes(uri, cursor=cursor, limit=100)
            likes = likes + fetched.likes

        except RequestException as e:
            print(e)
            cursor = old_cursor
            continue
        except HTTPError:
            return []
        except Exception as e:
            print(f"{datetime.datetime.now()} {e}")
            cursor = old_cursor
            continue

        if not fetched.cursor:
            break

        old_cursor = cursor
        cursor = fetched.cursor

    return likes


def clean_like(like):
    like = like.dict()
    who = like['actor']['handle']
    when = like['created_at']
    return who, when


def valid_time(t):
    try:
        T = datetime.datetime.fromisoformat(t)
        if T.date() > datetime.datetime(2024, 3, 18).date():
            return False
        elif T.date() < datetime.datetime(2023, 2, 17).date():
            return False
        return True
    except ValueError:
        return False
    except Exception as e:
        print(e)
        return None


The code below is used to run the edited version of the `crawl_feed_bookmarks.py` file.

**Note:** The cell below requires a long time to run!



In [ ]:
data = []
for f in feeds.feeds:
    if f.uri.strip().lower() in set(myfeeduris.values()):
        # print("inside loop")
        f = f.model_dump()
        data.append([f['display_name'], f['uri'], f['creator']['handle'], f['indexed_at'], f['description']])
        # break

df = pd.DataFrame(data, columns=['display_name', 'uri', 'creator', 'indexed_at', 'description'])
df.to_csv('feed_info_mine.csv', index=False, sep=';')

# who bookmarked a feed
with open('feed_likes_mine.csv', 'w') as f:
    for name, uri in myfeeduris.items():
        print(datetime.datetime.now(), name)
        likes = collect_likes(client, uri)
        for l in likes:
            who, when = clean_like(l)
            if valid_time(when):
                f.write(f"{name},{who},{when}\n")


2025-02-04 22:11:38.527662 Feed_1


KeyboardInterrupt: 

Handling `crawl_feed_post_likes.py`.

Original File:
*  Dependent on collection of `feed_uris`.
*  `feed_uris` here likely represents the unique identifiers for ALL posts on the platform.

Simplifying Assumption:


*   Used the feeds currently accessible from the popular_feeds_generator.
*   Our sample size right now is a lot smaller right now than it should be due to this assumption.

TO DO:


*   Dig through API to find a feed generator that sifts through all possible feeds.








NOTE: Cell below takes ~30 secs to run.

In [34]:
for f in feeds.feeds:
    feed_uri = normalize_uri(f.uri)
    unique_post_uris = set()

    try:
        posts = client.app.bsky.feed.get_feed(params={"feed":feed_uri, "limit":50})
        for post in posts.feed:
            post_uri = normalize_uri(post.post.uri)
            unique_post_uris.add(post_uri)

    except Exception as e:
        print(f"❌ Failed to fetch posts for {feed_uri}: {e}")

feed_uris_dict = {f"Post_{i+1}": uri for i, uri in enumerate(unique_post_uris)}


print(feed_uris_dict)

❌ Failed to fetch posts for at://did:plc:vc7f4oafdgxsihk4cry2xpze/app.bsky.feed.generator/media: Response(success=False, status_code=502, content=XrpcError(error='InvalidFeedResponse', message='feed provided an invalid response'), headers={'x-powered-by': 'Express', 'access-control-allow-origin': '*', 'cache-control': 'private', 'vary': 'Authorization, Accept-Encoding', 'ratelimit-limit': '3000', 'ratelimit-remaining': '2969', 'ratelimit-reset': '1738940840', 'ratelimit-policy': '3000;w=300', 'content-type': 'application/json; charset=utf-8', 'content-length': '77', 'etag': 'W/"4d-NKHU+i0HaghYl3XdGRanj9ADYRE"', 'date': 'Fri, 07 Feb 2025 15:02:35 GMT', 'keep-alive': 'timeout=90', 'strict-transport-security': 'max-age=63072000'})
❌ Failed to fetch posts for at://did:plc:2wqomm3tjqbgktbrfwgvrw34/app.bsky.feed.generator/authors: Response(success=False, status_code=502, content=XrpcError(error='InvalidFeedResponse', message='feed provided an invalid response'), headers={'x-powered-by': 'Exp

In [36]:
def save_feed_uris_to_file(post_uris, filename="feed_uris/popular_posts.txt.gz"):
    """
    Saves post URIs in a compressed .gz file for later processing.
    """
    os.makedirs("feed_uris", exist_ok=True)  # Ensure directory exists

    with gzip.open(filename, "wt", encoding="utf-8") as f:
        for uri in feed_uris_dict.values():
            f.write(uri + "\n")

    print(f"✅ Saved {len(post_uris)} post URIs to {filename}")

save_feed_uris_to_file(feed_uris_dict)

✅ Saved 36 post URIs to feed_uris/popular_posts.txt.gz


In [37]:
import gzip

file_path = "feed_uris/popular_posts.txt.gz"

with gzip.open(file_path, "rt", encoding="utf-8") as f:
    content = f.readlines()

print("📄 First 10 Saved Post URIs:")
print("".join(content[:10]))  # Print first 10 lines


📄 First 10 Saved Post URIs:
at://did:plc:o4gnmxjh5dyo7q3hpfpimczx/app.bsky.feed.post/3lhkddbzqmc2r
at://did:plc:462mkz4bspd3k4iuwzngv6eh/app.bsky.feed.post/3lhkeihcg3k2p
at://did:plc:rpty2ls6f4hmizzkajsnysb3/app.bsky.feed.post/3lhklqb4cys2y
at://did:plc:qjijic4bjkx2vbvb5ljhpaoa/app.bsky.feed.post/3lhkmvgf4f223
at://did:plc:ud4mbfwt7peg375w3x75vozp/app.bsky.feed.post/3lhkqbrf4fc2e
at://did:plc:osdix43tq2pdr46ixgvnx36l/app.bsky.feed.post/3lhk535lsz222
at://did:plc:gasv7nknoxqzcvqhizv5p55g/app.bsky.feed.post/3lhkncnfs7k25
at://did:plc:g5ij2v24pfmmnmgysowm3yuf/app.bsky.feed.post/3lhjindynac23
at://did:plc:mixdef3ii3valysmar4uhuln/app.bsky.feed.post/3lhlqc7j5u22n
at://did:plc:iylglgevqgtj26u3rrckgxib/app.bsky.feed.post/3lhlqqtgt3c2o



In [38]:
import gzip
import os
import datetime

from atproto_client import Client, SessionEvent
from atproto.exceptions import RequestException, BadRequestError

import datetime, time
import gzip
import os

# USERNAME = os.environ.get('USERNAME')
# PASSWORD = os.environ.get('PASSWORD')


import logging.handlers
log = logging.getLogger("bot")
log.setLevel(logging.DEBUG)
log.addHandler(logging.StreamHandler())


#### SESSION

def get_session():
    try:
        with open('session.txt') as f:
            return f.read()
    except FileNotFoundError:
        return None


def save_session(session_string):
    with open('session.txt', 'w') as f:
        f.write(session_string)


def on_session_change(event, session):
    print('Session changed:', event, repr(session))
    if event in (SessionEvent.CREATE, SessionEvent.REFRESH):
        print('Saving changed session')
        save_session(session.export())


# def init_client(USERNAME, PASSWORD):
#     client = Client()
#     client.on_session_change(on_session_change)

#     session_string = get_session()
#     if session_string:
#         print('Reusing session')
#         client.login(session_string=session_string)
#     else:
#         print('Creating new session')
#         client.login(USERNAME, PASSWORD)

#     return client

#### EXCP HANDLING

def sleep_until(when):
    now = datetime.datetime.now()
    when = datetime.datetime.fromtimestamp(when, datetime.UTC)
    if now.timestamp() > when.timestamp():
        pass
    else:
        print(f"waiting until {when}")
        time.sleep((when - now).total_seconds())


def _handle_requests_exceptions(e):
    status = e.response.status_code
    print(f"{datetime.datetime.now()}. error {status} {e.response.content.message}")
    if status == 429:  # too many
        when = int(e.response.headers['RateLimit-Reset'])
        sleep_until(when)

    elif status in {409, 413, 502}:  # net error
        time.sleep(50)
    else:
        pass


#### IO

def _save(likes, file_id):

    os.makedirs('feed_likes', exist_ok=True)

    with gzip.open(f'feed_likes/{file_id}.csv.gz', 'a') as f:
        for uri, post_author, ls in likes:
            for like in ls:
                user, created_at = clean_like(like)
                row = f'{user},{post_author},{uri},{created_at}\n'
                f.write(row.encode('utf8'))



def clean_like(like):
    like = like.dict()
    who = like['actor']['handle']
    when = like['created_at']
    return who, when


def collect_likes(client, uri, cursor=None, likes=None):
    cursor = None
    old_cursor = None

    if likes is None:
        likes = []


    while True:

        try:
            post_author = client.get_posts([uri]).posts[0].author.handle
            fetched = client.get_likes(uri, cursor=cursor, limit=100)
            likes = likes + fetched.likes

        except RequestException as e:
            _handle_requests_exceptions(e)
            cursor = old_cursor
            continue
        except BadRequestError:
            return None, []
        except Exception as e:
            print(f"{datetime.datetime.now()} {e}")
            cursor = old_cursor
            continue

        if not fetched.cursor:
            break

        old_cursor = cursor
        cursor = fetched.cursor

    return post_author, likes


# if __name__ == '__main__':


    # client = init_client(USERNAME, PASSWORD)


for file in os.listdir('feed_uris'):
    if file.endswith('.gz'):
        uris = []

        with gzip.open(os.path.join('feed_uris', file)) as f:
            feed_name = file.replace('.txt.gz', '')
            print('processing', feed_name)
            all_likes = []
            for l in f.readlines():
                uri = l.decode('utf8').rstrip()
                uris.append(uri)
                post_author, likes = collect_likes(client, uri)
                all_likes.append((uri, post_author, likes))
            _save(all_likes, feed_name)







processing popular_posts


<ipython-input-38-dd446664b0a3>:99: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  like = like.dict()


In [39]:
import gzip

file_path = "/content/feed_likes/popular_posts.csv.gz"

with gzip.open(file_path, "rt", encoding="utf-8") as f:
    content = f.readlines()

print("📄 File Contents:")
print("".join(content[:10]))

📄 File Contents:
sheyenne77.bsky.social,disposablewings.bsky.social,at://did:plc:o4gnmxjh5dyo7q3hpfpimczx/app.bsky.feed.post/3lhkddbzqmc2r,2025-02-07T02:33:31.035Z
disposablewings.bsky.social,disposablewings.bsky.social,at://did:plc:o4gnmxjh5dyo7q3hpfpimczx/app.bsky.feed.post/3lhkddbzqmc2r,2025-02-07T01:03:20.578Z
markhayden23.bsky.social,disposablewings.bsky.social,at://did:plc:o4gnmxjh5dyo7q3hpfpimczx/app.bsky.feed.post/3lhkddbzqmc2r,2025-02-07T00:51:26.650Z
kasper24.bsky.social,disposablewings.bsky.social,at://did:plc:o4gnmxjh5dyo7q3hpfpimczx/app.bsky.feed.post/3lhkddbzqmc2r,2025-02-07T00:22:23.870Z
astravelious.bsky.social,tinyteerex.bsky.social,at://did:plc:462mkz4bspd3k4iuwzngv6eh/app.bsky.feed.post/3lhkeihcg3k2p,2025-02-07T01:16:11.904Z
pastorkathleen.bsky.social,otheredones.bsky.social,at://did:plc:rpty2ls6f4hmizzkajsnysb3/app.bsky.feed.post/3lhklqb4cys2y,2025-02-07T13:33:01.106Z
mxfee.bsky.social,otheredones.bsky.social,at://did:plc:rpty2ls6f4hmizzkajsnysb3/app.bsky.feed.post/